In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice

In [2]:
def max_drawdown(returns):
        local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or 
           ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))] 
        
        local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
                (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]
        
        def next_local_min(n):
            if [m for m in local_min if m > n]:
                return [m for m in local_min if m > n][0]
            else: return None
        
        drawdowns = [(n,next_local_min(n)) for n in local_max]
        drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
        if drawdown_values:
            return  np.max(drawdown_values)
        else: return 0.0

In [ ]:
# 1. Pull in list of available ATM puts and calls with 30-day expiry
# 2. Compare IV against our calculated IV, generating a new column 'trade' with values -1, 0, 1 depending if we go short, neutral, or long.
# 3. Pull in simulation csv based on long/short (think this can be improved, just put it all into 1 CSV after determining 
    # short/neutral/long in the simulation function
# 4. Allocate some amount of capital, let's say we start with 10MM, to each trade based on some kind of confidence calculation 
    # ([Our IV  - IV] / threshold * max[min[5% of the portfolio, 80% of cash], 100% of cash]) 
#     threshold = median or some percentile of [Our IV - IV]
# 5. Have 'cash' column that starts with 10MM (have it earn interest?)
# 6. Calculate 'capital_allocated', options only trade in whole contracts, so round to nearest contract count, multiply by total cost to enter
# 7. Keep a 'gross_trading_value' column (of options, shares can be netted at each date) to calculate trading costs in future
# 8. 'capital_allocated' column that corresponds with each of the available ATM puts/calls with 30-day expiry, multiply by net value to enter (stock + option)
# 9. Multiply unit PnL for each position by 'capital_allocated' to get PnL for that position
# 10. Set a drawdown limit (10%? will have to look at data) for PnL of each, determine whether to exit early
# 11. Net change in stocks for each date, add this change to 'gross_trading_value'
# 12. PnL final

In [ ]:
# 2 dataframes, one with all positions, the other for PnL at each date (have gross trading costs in this one)

In [4]:
def pos_limit(df):
    asdf = 0